#### In the previous notebook<a href="https://www.kaggle.com/code/yunsuxiaozi/hms-learn-to-transform-eeg-to-spectrogram-256-256">HMS Learn to transform eeg to spectrogram(256*256)</a>,I learned how to use eeg data to generate spectrograms. In this notebook, I will use these data to train the model.

### Import necessary libraries

In [ ]:
#https://www.kaggle.com/code/ttahara/hms-hbac-resnet34d-baseline-training
#https://www.kaggle.com/code/ttahara/hms-hbac-resnet34d-baseline-inference
#necessary
import pandas as pd#导入csv文件的库
import numpy as np#进行矩阵运算的库
import matplotlib.pyplot as plt#导入强大的绘图库
import torch #一个深度学习的库Pytorch
import timm#图像分类预训练模型库
import torch.nn as nn#neural network,神经网络
import torch.optim as optim#一个实现了各种优化算法的库
import torch.nn.functional as F#神经网络函数库
import torchvision.transforms as transforms#Pytorch下面的图像处理库,用于对图像进行数据增强
#设置随机种子
import random
import warnings#避免一些可以忽略的报错
warnings.filterwarnings('ignore')#filterwarnings()方法是用于设置警告过滤器的方法，它可以控制警告信息的输出方式和级别。

### Config

In [ ]:
class Config:
    seed=2024
    image_transform=transforms.Resize((512,512))
    batch_size=64
    num_epochs=10#由于是预训练模型,训练10个epoch足够了
    num_folds=5#5折交叉验证.

### Set seed

In [ ]:
def seed_everything(seed):
    torch.backends.cudnn.deterministic = True#将cuda加速的随机数生成器设为确定性模式
    torch.backends.cudnn.benchmark = True#关闭CuDNN框架的自动寻找最优卷积算法的功能，以避免不同的算法对结果产生影响
    torch.manual_seed(seed)#pytorch的随机种子
    np.random.seed(seed)#numpy的随机种子
    random.seed(seed)#python内置的随机种子
seed_everything(Config.seed)

### Import dataset

In [ ]:
train_df=pd.read_csv("/kaggle/input/hms-harmful-brain-activity-classification/train.csv")

labels=['seizure','lpd','gpd','lrda','grda','other']
for label in labels:
    group=train_df[f'{label}_vote'].groupby(train_df['eeg_id']).sum()
    label_vote_sum = pd.DataFrame({'eeg_id': group.index, f'{label}_vote_sum': group.values})
    if label=='seizure':
        train_feats=label_vote_sum
    else:
        train_feats=train_feats.merge(label_vote_sum,on='eeg_id',how='left')
train_feats['total_vote']=0
for label in labels:
      train_feats['total_vote']+=train_feats[f'{label}_vote_sum']
for label in labels:
      train_feats[f'{label}_vote']=train_feats[f'{label}_vote_sum']/train_feats['total_vote']
choose_cols=['eeg_id']
for label in labels:
    choose_cols+=[f'{label}_vote']
train_feats=train_feats[choose_cols]
train_feats['path']=train_feats['eeg_id'].apply(lambda x: "/kaggle/input/hms-harmful-brain-activity-classification/train_eegs/"+str(x)+".parquet" )
train_npy=np.load("/kaggle/input/hms-learn-to-transform-eeg-to-spectrogram-256-256/eeg_specs.npy",allow_pickle=True).item()
train_feats.head()

### Metric

In [ ]:
def KL_loss(p,q):
    epsilon=10**(-15)
    p=torch.clip(p,epsilon,1-epsilon)
    q = nn.functional.log_softmax(q,dim=1)
    #对第一个维度,就是num_classes维度的损失求和,得到每个样本的损失,然后对第0维求平均,得到每个样本平均KL散度.
    return torch.mean(torch.sum(p*(torch.log(p)-q),dim=1))

### get batch_data

In [ ]:
def get_batch(paths,batch_size=Config.batch_size):
    eps=1e-6
    batch_data=[]
    for path in paths:
        data = train_npy[int(path[0][67:-8])]
        data_tensor = torch.unsqueeze(torch.Tensor(data), dim=0)
        data=Config.image_transform(data_tensor)
        batch_data.append(data)
    batch_data=torch.stack(batch_data)
    return batch_data

### Model training

In [ ]:
device ='cuda' if  torch.cuda.is_available() else 'cpu'
print(f"device:{device}")


total_idx=np.arange(len(train_feats))
np.random.shuffle(total_idx)

for fold in range(Config.num_folds):

    test_idx=total_idx[fold*len(total_idx)//Config.num_folds:(fold+1)*len(total_idx)//Config.num_folds]
    train_idx=np.array([idx for idx in total_idx if idx not in test_idx])

    #下载模型resnet34d,并且将训练好的参数加载进来.每折初始化一个模型.
    model = timm.create_model('resnet34d',pretrained=True, num_classes=6, in_chans=1)
    
    #优化器
    optimizer=optim.AdamW(model.parameters(),lr=0.001,betas=(0.5,0.999),weight_decay=0.01)

    best_test_loss=1.0#目前损失最好的成绩,每一折都要重新初始化
    train_losses=[]
    test_losses=[]

    print(f"start training fold:{fold}")

    for epoch in range(Config.num_epochs):
        print(f"epoch {epoch}:")
        model.to(device)#保存模型的时候可能会转成CPU,故设置这个来保证模型会在GPU上训练.
        #训练
        model.train()
        train_loss=[]
        random_num=np.arange(len(train_idx))
        np.random.shuffle(random_num)
        train_idx=train_idx[random_num]

        for idx in range(0,len(train_idx),Config.batch_size): 
            #将梯度清空
            optimizer.zero_grad()
            train_idx1=train_idx[idx:idx+Config.batch_size]
            train_X1_path=train_feats[['path']].iloc[train_idx1].values
            train_X1=get_batch(train_X1_path,batch_size=Config.batch_size)
            train_y1=train_feats[['seizure_vote','lpd_vote','gpd_vote','lrda_vote','grda_vote','other_vote']].iloc[train_idx1].values
            train_y1=torch.Tensor(train_y1)
            #将数据放进去训练
            train_pred=model(train_X1.to(device)).to(device)
            #计算每次的损失函数
            loss=KL_loss(train_y1.to(device),train_pred.to(device)).to(device)
            #反向传播
            loss.backward()
            #优化器进行优化(梯度下降,降低误差)
            optimizer.step()
            train_loss.append(loss.detach().cpu().numpy())
        train_loss=np.mean(np.array(train_loss))
        print(f"train_loss:{train_loss}")
        test_loss=[]
        model.eval()
        with torch.no_grad():
            for idx in range(0,len(test_idx),Config.batch_size): 
                test_idx1=test_idx[idx:idx+Config.batch_size]
                test_X1_path=train_feats[['path']].iloc[test_idx1].values
                test_X1=get_batch(test_X1_path,batch_size=Config.batch_size)

                test_y1=train_feats[['seizure_vote','lpd_vote','gpd_vote','lrda_vote','grda_vote','other_vote']].iloc[test_idx1].values
                test_y1=torch.Tensor(test_y1)
                #将数据放进去训练
                test_pred=model(test_X1.to(device)).to(device)
                #计算每次的损失函数
                loss=KL_loss(test_y1.to(device),test_pred.to(device)).to(device)
                test_loss.append(loss.detach().cpu().numpy())
        test_loss=np.mean(np.array(test_loss))
        print(f"test_loss:{test_loss}")
        if test_loss<best_test_loss:#如果这折的这个模型的这次训练比上次好,保存起来
            best_test_loss=test_loss
            torch.save(model.to('cpu'),f"HMS_resnet34d_fold{fold}.pth")
            
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        print("-"*50)
    print(f"best_test_loss:{best_test_loss}") 
    plt.title("train_losses VS test_losses")
    epochs=[i for i in range(len(train_losses))]
    plt.plot(epochs,train_losses,marker="o",markersize=1,label="train_losses")
    plt.plot(epochs,test_losses,marker="x",markersize=1,label="test_losses")
    plt.legend()
    plt.show()